In [1]:
%reload_ext yamlmagic

In [2]:
%%yaml starting_links
- https://github.com/bollwyvl/jademagic
- http://nbviewer.jupyter.org/urls/gist.githubusercontent.com/tonyfast/62e0d05a85c06f1938d1f8c17cef6dee/raw/2bd7f1d033d05c8c9a6741e47f0fb3eda26eedbe/mdmagic.ipynb

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
import jinja2
jinja2.__version__

'2.8'

In [24]:
%reload_ext jademagic

In [85]:
%%file markdownmagic.py
from __future__ import print_function

from IPython import get_ipython

import IPython

from IPython.core import magic_arguments

from IPython.core.magic import (
    Magics,
    magics_class,
    cell_magic,
)

from jinja2 import (
    Environment,
    DictLoader,
)

import mistune
from mistune import (
    Renderer,
)

import yaml

__version_info__ = (0, 1, 0)
__version__ = '.'.join(map(str, __version_info__))


class LiterateRenderer(Renderer):
    """Executes code for on Markdown code fences for the language ``python``"""
    def __init__(self, ip, *args, **kwargs):
        self.ip = ip
        super().__init__(*args, **kwargs)

    def block_code(self, text, lang):
        if lang in ['python']:
            self.ip.run_code(text)
        return super().block_code(text, lang)


class MarkdownerTemplate(IPython.core.display.Markdown):
    def __init__(
        self,
        env=Environment(),
        ip=get_ipython(),
        frontmatter="""""",
        *args,
        **kwargs
    ):
        self.env = env
        self.ip = ip
        self.frontmatter = frontmatter
        super().__init__(*args, **kwargs)
        if self.frontmatter:
            self.frontmatter = yaml.load(
                self.env.from_string(
                    self.frontmatter
                ).render(self.ip.user_ns)
            )
        else:
            self.frontmatter = {}
        self.template = self.env.from_string(self.data)

    def __add__(self, payload):
        if payload in ['*']:
            payload = self.ip.user_ns
        return self.template.render(**payload)

    def __mul__(self, payload):
        return '\n'.join([self.template.render(**load) for load in payload])

    def _repr_markdown_(self):
        self.env.filters['mistune_renderer'](self.data)
        return self + {**self.frontmatter, **self.ip.user_ns}


@magics_class
class environment(Magics):
    """
    Literate Python in using Markdown.

    Example:

        %%markdown
        # This is a title with {{data}}
    """
    ip = get_ipython()

    def __init__(self, env=Environment(loader=DictLoader({}))):
        self.env = env
        self.env.filters['mistune_renderer'] = mistune.Markdown(
            renderer=LiterateRenderer(ip=self.ip, escape=False)
        )
        super().__init__()
        self.ip.register_magics(self)

    @cell_magic
    @magic_arguments.magic_arguments()
    @magic_arguments.argument(
        "name",
        default=None,
        nargs="?",
        help="""Name of local variable to set to parsed value"""
    )
    def markdown(self, line, cell):
        line = line.strip()
        args = magic_arguments.parse_argstring(self.markdown, line)
        if cell.startswith('---'):
            frontmatter, cell = cell.lstrip('---').split('---', 1)
        else:
            frontmatter = {}
        try:
            # Literate python execution
            display = MarkdownerTemplate(
                self.env,
                self.ip,
                frontmatter,
                data=cell
            )
        except Exception as err:
            print(err)
            return

        if args.name:
            self.ip.user_ns[args.name] = display
        return display



Overwriting markdownmagic.py


In [86]:
!pwd

/Users/tfast/jademagic


# Go through files one by one and make changes.

- ``Automation.ipynb`` - * no changes *
- ``LICENSE`` - * change to my name*
- ``MANIFEST.in`` - * no changes *
- ``README.md`` - Made it relevent to markdownamagic
- ``requirements.txt`` - import jinja2, mistune, and yamlmagic
- ``setup.py`` - Change metadata and personal dataa

> Copy Markdown Magic from the Notebook

In [ ]:
"""These code below creates the markdown magics class."""
import mistune, IPython, jinja2

class LiterateRenderer(mistune.Renderer):
    """Executes code for on Markdown code fences for the language ``python``"""
    def __init__( self, ip, *args, **kwargs):
        self.ip = ip
        super().__init__(*args, **kwargs)

    def block_code( self, text, lang ):
        if lang in ['python']:
            self.ip.run_code(text)
        return super().block_code( text, lang )

class MarkdownerTemplate( IPython.core.display.Markdown ):
    import yaml
    def __init__( self, env = jinja2.Environment(), ip=IPython.get_ipython(), frontmatter="""""", *args, **kwargs):
        self.env = env
        self.ip = ip
        self.frontmatter = frontmatter
        super().__init__(*args, **kwargs)
        if self.frontmatter:
            self.frontmatter = self.yaml.load(self.env.from_string( self.frontmatter ).render( self.ip.user_ns ))
        else:
            self.frontmatter = {}
        self.template = self.env.from_string( self.data )

    def __add__( self, payload ):
        if payload in ['*']:
            payload = self.ip.user_ns
        return self.template.render( **payload )
    def __mul__( self, payload ):
        return '\n'.join([self.template.render(**load) for load in payload ])
        
    def _repr_markdown_( self ):
        mistune.markdown( self.data, renderer = LiterateRenderer(ip=self.ip,escape=False) )
        return self + {**self.frontmatter, **self.ip.user_ns}

@IPython.core.magic.magics_class
class MarkdownMagic(IPython.core.magic.Magics):
    import jinja2, yaml
    ip = get_ipython()    
    def __init__( self, env = self.jinja2.Environment(loader=jinja2.DictLoader({})) ):
        self.env = env
        super().__init__()
        self.ip.register_magics(self)
        
    @IPython.core.magic.cell_magic
    @IPython.core.magic_arguments.magic_arguments()
    @IPython.core.magic_arguments.argument( "name", default="""""", nargs="?", help="""Name of local variable to set to parsed value""")
    def markdown(self, line, cell):
        line = line.strip()
        args = IPython.core.magic_arguments.parse_argstring(self.markdown, line)
        if cell.startswith('---'):
            frontmatter, cell = cell.lstrip('---').split('---',1)
        else: 
            frontmatter = {}
        display = MarkdownerTemplate( self.env, self.ip, frontmatter, data=cell ) # Literate python execution
        if args.name:
            self.ip.user_ns[args.name] = display
        return display